### Assignment 9

In [1]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }a
</style>
"""))

In [2]:
import pandas as pd
#from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt 
import chardet

import folium as fm
from folium import Marker, GeoJson
from folium.plugins import MarkerCluster, HeatMap, StripePattern

import geopandas as gpd
from geopandas import GeoSeries
from shapely.geometry import Point, LineString


import branca as br 

### 1. Generate a Choropleth map of the total institutes' vacancies by the department.

In [3]:
inst = pd.read_csv(r'../../_data/institutos1.csv')

inst_geo = gpd.GeoDataFrame (inst, crs = "EPSG:4326",
                            geometry = gpd.points_from_xy( inst.nlong_ie,
                                                         inst.nlat_ie))
shp_dpt = gpd.read_file(r"..\..\_data\INEI_LIMITE_DEPARTAMENTAL\INEI_LIMITE_DEPARTAMENTAL.shp")

intersect_inst_geo = gpd.overlay(inst_geo,shp_dpt, how = 'intersection')

vars_sum = ['ltimoden_metaatencion', 'cuentadeid_postulante_procesoadm',
           'sumaden_flagingresante','sumaden_flagmatriculado']
tot_dpt = intersect_inst_geo.groupby(['CCDD'], as_index = False)[vars_sum].sum()

tot_dpt_shp = shp_dpt.merge(tot_dpt, on = 'CCDD')

In [43]:
dpt1 = tot_dpt_shp[['CCDD', 'geometry']]
dpt1

,CCDD,geometry
0,01,"POLYGON ((-77.81399 -2.99278, -77.81483 -2.995..."
1,02,"POLYGON ((-77.64697 -8.05086, -77.64689 -8.051..."
2,03,"POLYGON ((-73.74655 -13.17442, -73.74570 -13.1..."
3,04,"POLYGON ((-71.98109 -14.64062, -71.98093 -14.6..."
4,05,"POLYGON ((-74.34843 -12.17503, -74.35000 -12.1..."
5,06,"POLYGON ((-78.22182 -7.76346, -78.22233 -7.763..."
6,07,"POLYGON ((-77.13504 -11.81820, -77.13484 -11.8..."
7,08,"POLYGON ((-72.97280 -11.25189, -72.97134 -11.2..."
8,09,"POLYGON ((-74.57118 -11.98710, -74.57095 -11.9..."
9,10,"POLYGON ((-75.99200 -8.32108, -75.99015 -8.321..."


TypeError: 'tuple' object is not callable

In [47]:
# government palace coordinates

lat_palacio = -12.0757538
long_palacio = -76.9863174
zoom_start = 5

z = fm.Map(location = [lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start = zoom_start)

# Mandatory: geo_data in GeoJson format
# columns: variables from economics indicators data set
# Atention !!! key_on: commom variable between geodata and data "feature.properties.(name of variable)"

fm.Choropleth(
    geo_data=dpt1,
    data=tot_dpt_shp,
    columns=['ltimoden_metaatencion'],
    key_on="feature.properties.CCDD",
    fill_color="YlOrRd",
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name="Total vacancies",
    smooth_factor=0,
    Highlight= True,
    line_color = "#0000",
    overlay=True,
    nan_fill_color = "White"  # fill white missing values 
    ).add_to(z)

#fm.LayerControl().add_to(z)

# Save in a html format 

#z.save("Poverty_Map.html")

z

IndexError: list index out of range

In [4]:
inst

,cod_mod,ltimoden_metaatencion,cuentadeid_postulante_procesoadm,sumaden_flagingresante,sumaden_flagmatriculado,ratio,ratio1,dif,dif1,nlat_ie,...,altitude,Dblock1km,Dblock2km,Dblock3km,Dblock5km,Dblock10km,Dblock20km,Dblock30km,_merge,geometry
0,207597,307,481,302,301,0.980456,1.566775,-6,174,-13.52107,...,3386.0,14840369,742184,494123,296073,148036,7418,4912,matched (3),POINT (-71.97748 -13.52107)
1,207613,240,502,204,204,0.850000,2.091667,-36,262,-13.63997,...,2342.0,14970470,748235,499156,299094,149047,7423,4915,matched (3),POINT (-72.88261 -13.63997)
2,239970,537,1047,484,476,0.886406,1.949721,-61,510,-15.47827,...,3831.0,17000163,850081,566054,340032,170016,8508,5605,matched (3),POINT (-70.12756 -15.47827)
3,262311,423,526,353,349,0.825059,1.243499,-74,103,-6.23041,...,2358.0,6781025,339512,226341,135205,67102,3351,2234,matched (3),POINT (-77.86943 -6.23041)
4,273979,425,544,411,405,0.952941,1.280000,-20,119,-6.49004,...,309.0,7070857,353428,235285,141171,70085,3542,2328,matched (3),POINT (-76.36539 -6.49004)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,1468594,1566,250,241,202,0.128991,0.159642,-1364,-1316,-12.02987,...,125.0,13190935,659467,439311,263187,131093,6546,4331,matched (3),POINT (-77.05831 -12.02987)
211,1468610,284,74,71,71,0.250000,0.260563,-213,-210,-12.08215,...,115.0,13250934,662467,441311,265186,132093,6646,4431,matched (3),POINT (-77.05295 -12.08215)
212,1545623,210,266,184,177,0.842857,1.266667,-33,56,-16.13940,...,3780.0,17740208,887104,591069,354041,177020,8810,5906,matched (3),POINT (-70.53580 -16.13940)
213,1595495,206,144,141,137,0.665049,0.699029,-69,-62,-18.01370,...,577.0,19810177,990088,660059,396035,198017,9908,6605,matched (3),POINT (-70.25080 -18.01370)


In [5]:
inst.columns

Index(['cod_mod', 'ltimoden_metaatencion', 'cuentadeid_postulante_procesoadm',
       'sumaden_flagingresante', 'sumaden_flagmatriculado', 'ratio', 'ratio1',
       'dif', 'dif1', 'nlat_ie', 'nlong_ie', 'population_ccpp', 'altitude',
       'Dblock1km', 'Dblock2km', 'Dblock3km', 'Dblock5km', 'Dblock10km',
       'Dblock20km', 'Dblock30km', '_merge', 'geometry'],
      dtype='object')

## 2. Make an interactive map of markes using visual_html function and show total vacancies, applicants, entrants, and enrolled information in a table popup. Additionally, use "university" as icon.

In [6]:
def visual_html(i):
   
    Total_Vacancies = inst['ltimoden_metaatencion'].iloc[i]                            
    Total_Applicants = inst['cuentadeid_postulante_procesoadm'].iloc[i]
    Total_Entrants = inst['sumaden_flagingresante'].iloc[i]  
    Total_Enrolled = inst['sumaden_flagmatriculado'].iloc[i]  
    
    
    left_col_colour = "#FA8072"
    right_col_colour = "#BDC3C7"
    
    html = """<!DOCTYPE html>
<html>

<head>
    <p> Institutes </p>

</head>
    <table style="height: 126px; width: 350px;">  <!-- Comment: Create a teable. -->

<!-- Add information  -->

<tbody> 
<tr>

<!-- Add color by column -->

<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Total Vacancies</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(Total_Vacancies) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Total Applicants</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(Total_Applicants) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Total Entrants</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(Total_Entrants) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Total Enrolled</span></td>
<td style="width: 150px;background-color: """+ right_col_colour +""";">{}</td>""".format(Total_Enrolled) + """
</tr>


</tbody>
</table>
</html>
"""
    return html

In [7]:
ubication = inst['nlat_ie'].mean(), inst['nlong_ie'].mean()  # Average point

inst2 = fm.Map(location = ubication, zoom_start=12)

for i in range(0,len(inst)):
    html = visual_html(i)

    iframe = br.element.IFrame(html=html,width=350,height=300)
    popup = fm.Popup(iframe, parse_html=True)
    
    fm.Marker([inst['nlat_ie'].iloc[i],inst['nlong_ie'].iloc[i]],
                  popup=popup, icon=fm.Icon(color= 'blue', icon='university', prefix="fa")).add_to(inst2)

inst2.save("institutes.html")
inst2

## Generate the ratio of applicants by vacancies. Then, generate a cluster map with two groups of marker clusters: Applicant Surplus and Applicant Deficit.

In [8]:
inst.dtypes

cod_mod                                int64
ltimoden_metaatencion                  int64
cuentadeid_postulante_procesoadm       int64
sumaden_flagingresante                 int64
sumaden_flagmatriculado                int64
ratio                                float64
ratio1                               float64
dif                                    int64
dif1                                   int64
nlat_ie                              float64
nlong_ie                             float64
population_ccpp                      float64
altitude                             float64
Dblock1km                              int64
Dblock2km                              int64
Dblock3km                              int64
Dblock5km                              int64
Dblock10km                             int64
Dblock20km                             int64
Dblock30km                             int64
_merge                                object
geometry                            geometry
dtype: obj

In [9]:
inst['ratio_applicants_vacancies'] = (inst['cuentadeid_postulante_procesoadm']/inst['ltimoden_metaatencion'])
inst

,cod_mod,ltimoden_metaatencion,cuentadeid_postulante_procesoadm,sumaden_flagingresante,sumaden_flagmatriculado,ratio,ratio1,dif,dif1,nlat_ie,...,Dblock1km,Dblock2km,Dblock3km,Dblock5km,Dblock10km,Dblock20km,Dblock30km,_merge,geometry,ratio_applicants_vacancies
0,207597,307,481,302,301,0.980456,1.566775,-6,174,-13.52107,...,14840369,742184,494123,296073,148036,7418,4912,matched (3),POINT (-71.97748 -13.52107),1.566775
1,207613,240,502,204,204,0.850000,2.091667,-36,262,-13.63997,...,14970470,748235,499156,299094,149047,7423,4915,matched (3),POINT (-72.88261 -13.63997),2.091667
2,239970,537,1047,484,476,0.886406,1.949721,-61,510,-15.47827,...,17000163,850081,566054,340032,170016,8508,5605,matched (3),POINT (-70.12756 -15.47827),1.949721
3,262311,423,526,353,349,0.825059,1.243499,-74,103,-6.23041,...,6781025,339512,226341,135205,67102,3351,2234,matched (3),POINT (-77.86943 -6.23041),1.243499
4,273979,425,544,411,405,0.952941,1.280000,-20,119,-6.49004,...,7070857,353428,235285,141171,70085,3542,2328,matched (3),POINT (-76.36539 -6.49004),1.280000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,1468594,1566,250,241,202,0.128991,0.159642,-1364,-1316,-12.02987,...,13190935,659467,439311,263187,131093,6546,4331,matched (3),POINT (-77.05831 -12.02987),0.159642
211,1468610,284,74,71,71,0.250000,0.260563,-213,-210,-12.08215,...,13250934,662467,441311,265186,132093,6646,4431,matched (3),POINT (-77.05295 -12.08215),0.260563
212,1545623,210,266,184,177,0.842857,1.266667,-33,56,-16.13940,...,17740208,887104,591069,354041,177020,8810,5906,matched (3),POINT (-70.53580 -16.13940),1.266667
213,1595495,206,144,141,137,0.665049,0.699029,-69,-62,-18.01370,...,19810177,990088,660059,396035,198017,9908,6605,matched (3),POINT (-70.25080 -18.01370),0.699029


In [10]:
A_surplus = inst[inst['ratio_applicants_vacancies']>=1].reset_index(drop=True)
A_deficit = inst[inst['ratio_applicants_vacancies']<1].reset_index(drop=True)

In [11]:
surplus = list(zip(A_surplus['nlat_ie'], A_surplus['nlong_ie']))
deficit = list(zip(A_deficit['nlat_ie'], A_deficit['nlong_ie']))

In [12]:
lat_palacio = -12.0757538
long_palacio = -76.9863174

# List of tiles 
inst3 = fm.Map(location = [lat_palacio, long_palacio], zoom_start = 11)

# Cluster Map
MarkerCluster( surplus, name = 'Applicant Surplus' ).add_to(inst3)
MarkerCluster( deficit, name = 'Applicant Deficit' ).add_to(inst3)

fm.LayerControl().add_to(inst3)

inst3.save("cluster_aplicants_vacancies.html")

inst3

## 4 Generate a Circle map and plot only institutes with more applicants than vacancies. Make the radius= ratio_applicants_vacancies * 10000 . Make these circles of color green. Name the layer of circles as Applicant Surplus. Use FeatureGroup

In [35]:
# Genereate groups in folium

inst4 = fm.Map(location = [lat_palacio, long_palacio], zoom_start = 11)

institutesurplus = fm.FeatureGroup(name = 'Applicant Surplus', control=True)

tooltip = "Click"

for i, row in A_surplus.iterrows():
    
    fm.Circle([row['nlat_ie'], row['nlong_ie']], popup=row['cod_mod'],
              radius = 10000,
              fill = True,
              color="green",
              fill_color="green",
              tooltip=tooltip).add_to(inst4)
                        
                        
inst4

### 5. Generate a Circle map and plot only institutes with less applicants than vacancies. Make the radius= ratio_applicants_vacancies * 10000 . Make these circles of color red. Name the layer of circles as Applicant Deficit. Use FeatureGroup.

In [36]:
# Genereate groups in folioum
inst5 = fm.Map(location = [lat_palacio, long_palacio], zoom_start = 11)

institutedeficit = fm.FeatureGroup( name = 'Applicant Deficit', control=True)

tooltip = "Click"

for i, row in A_deficit.iterrows():
    
    fm.Circle([row['nlat_ie'], row['nlong_ie']], popup=row['cod_mod'],
              radius = 10000,
              fill = True,
              color="red",
              fill_color="red",
              tooltip=tooltip).add_to(inst5)
                        
                        
inst5

## 6. Generate a Circle Map that combines map 4 and 5 into a unique map. Use FeatureGroup. Additionally, generate a legend for this map indicating the color and label.

In [46]:
base1 = A_surplus
base2 = A_deficit

inst6 = fm.Map(location = [lat_palacio, long_palacio], zoom_start = 12.5, control_scale=True)

colors = ['green','red']

for j in range(1,3):
        
    for idx, row in globals()[f'base{j}'].iterrows():
        fm.Circle([row['nlat_ie'], row['nlong_ie']], radius = 10000, color = colors[j-1], legend_name = "Ration Applicant Vacacies").add_to(inst6)

inst6